## Importação de Bibliotecas

In [9]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import cv2
import requests
from bs4 import BeautifulSoup
import io
import numpy as np
from PIL import Image

## Funções Auxiliares

### Função para validar documentos com detecção de rosto

In [10]:
def validate_document(file_content):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img = Image.open(io.BytesIO(file_content))
    img_array = np.array(img)
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    return len(faces) > 0

### Função para validar links de perfis

In [11]:
def validate_profile_link(link, interests):
    try:
        response = requests.get(link, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text().lower()
        return any(interest.lower() in text for interest in interests.split(','))
    except:
        return False

## Seções do formulário: 

### Seção 1: Dados Pessoais

In [12]:
name_input = widgets.Text(description="Nome:")
address_input = widgets.Text(description="Endereço:")
cpf_input = widgets.Text(description="CPF:")
display(name_input, address_input, cpf_input)

Text(value='', description='Nome:')

Text(value='', description='Endereço:')

Text(value='', description='CPF:')

### Seção 2: Interesses

In [13]:
interests_input = widgets.Textarea(description="Interesses (ex: jogos, times):", rows=3)
display(interests_input)

Textarea(value='', description='Interesses (ex: jogos, times):', rows=3)

### Seção 3: Upload de Documento

In [14]:
doc_upload = widgets.FileUpload(description="Upload de Documento", accept="image/*")
doc_output = widgets.Output()
display(doc_upload, doc_output)

def on_doc_upload_change(change):
    with doc_output:
        clear_output()
        if doc_upload.value:
            file_info = list(doc_upload.value.values())[0]
            if validate_document(file_info['content']):
                print("Validando documento com IA... Validação concluída com sucesso!")
            else:
                print("Validando documento com IA... Nenhum rosto detectado.")
doc_upload.observe(on_doc_upload_change, names='value')

FileUpload(value=(), accept='image/*', description='Upload de Documento')

Output()

### Seção 4: Redes Sociais

In [7]:
social_input = widgets.Text(description="Redes Sociais (ex: @usuario):")
social_output = widgets.Output()
display(social_input, social_output)

def on_social_change(change):
    with social_output:
        clear_output()
        if social_input.value:
            print(f"Simulando leitura de interações... Encontradas 50 interações relacionadas a esports.")
social_input.observe(on_social_change, names='value')

Text(value='', description='Redes Sociais (ex: @usuario):')

Output()

### Seção 5: Links de Perfis

In [8]:
profile_link_input = widgets.Text(description="Link de Perfil:")
profile_output = widgets.Output()
display(profile_link_input, profile_output)

def on_link_change(change):
    with profile_output:
        clear_output()
        if profile_link_input.value and validate_profile_link(profile_link_input.value, interests_input.value):
            print("Validando conteúdo com IA... Conteúdo relevante ao perfil!")
        elif profile_link_input.value:
            print("Validando conteúdo com IA... Conteúdo não relevante ou link inválido.")
profile_link_input.observe(on_link_change, names='value')

Text(value='', description='Link de Perfil:')

Output()

## Instruções de Uso

1. **Dados Pessoais**: Insira seu nome, endereço e CPF nos campos fornecidos.
2. **Interesses**: Descreva seus interesses em esports, como jogos favoritos, times (ex: FURIA), eventos e compras relacionadas.
3. **Upload de Documento**: Faça upload de uma imagem de um documento de identificação. A IA detectará se há um rosto na imagem.
4. **Redes Sociais**: Insira seu perfil de rede social (ex: Twitter, Instagram). A simulação exibirá interações relacionadas a esports.
5. **Links de Perfis**: Insira um link para seu perfil em um site de esports. A IA verificará se o conteúdo é relevante com base nos seus interesses.